In [ ]:
!pip install selenium
!apt install chromium-chromedriver

     |████████████████████████████████| 963 kB 5.1 MB/s 
     |████████████████████████████████| 359 kB 40.4 MB/s 
     |████████████████████████████████| 138 kB 42.5 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 3.6 MB 34.9 MB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following pack

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 5.1 MB/s 
     |████████████████████████████████| 6.8 MB 28.6 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 895 kB 22.9 MB/s 
     |████████████████████████████████| 596 kB 46.7 MB/s 
     |████████████████████████████████| 127 kB 52.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.8
    Uninstalling urllib3-1.26.8:
      Successfully uninstalled urllib3-1.26.8
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.1.2 requires urllib3[secure,socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.

In [ ]:
import sys
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm
import os
import re
import time
from collections import Counter

import transformers
from transformers import BertTokenizer,AdamWeightDecay,TFRobertaModel,TFBertModel

import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping,ModelCheckpoint

import sklearn
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import StratifiedKFold

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/22-1 nlp 스터디/'
train = pd.read_csv(path + 'train_data.csv')
test = pd.read_csv(path+'test_data.csv')
submission = pd.read_csv(path+ "sample_submission.csv")
topic_dict = pd.read_csv(path+'topic_dict.csv')

In [ ]:
def chrome_setting():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=chrome_options)
  return driver

In [ ]:
data = pd.read_csv(path+'data/kr_train_10000_19999_ver2.csv')

In [ ]:
data

,index,title,topic_idx,topic_name,eng_title
0,10000,19시간 조사받은 손석희·김웅…폭행·협박 놓고 평행선,2,사회,Son Seokhee and Kim Woong who were surveyed fo...
1,10001,진중권이 돌아보는 조영남 그림 대작 사건,3,생활문화,The masterpiece of Cho Youngnam's painting tha...
2,10002,靑안보실 차장 비공개 방미…美일각 선제타격론 제기속 주목,6,정치,Deputy Director of the Blue House Security Off...
3,10003,마스크·우산 챙기세요 미세먼지에 빗방울도,3,생활문화,Make sure to bring a mask and an umbrella. Fin...
4,10004,kt 연패 탈출 1등 공신 내쉬 아주 절실한 승리였다,5,스포츠,The first place contributor to KT's consecutiv...
...,...,...,...,...,...
9995,19995,황총리 IS 테러가능성 있는 시설물 테러대비 태세 점검,6,정치,Prime Minister Hwang checks the readiness for ...
9996,19996,野 국민통합 위한 특사돼야…신중한 검토 필요,6,정치,It should be a special envoy for national inte...
9997,19997,미 국무부 힐러리 이메일 881건 추가공개…88건 기밀,4,세계,U.S. Department of State Hillary's 881 additio...
9998,19998,민간 위성개발 시대 열린다…KAI 중형위성 2호기 개발 주관,0,IT과학,The era of civilian satellite development will...


In [ ]:
data['eng_title'][8001]

'The truth of 15 years ago that an old lover encounters...A round trip to a play.'

In [ ]:
driver=chrome_setting()

In [ ]:
def trans_to_kor(transed_list, transed_lang,start_index,final_index): 
  
  target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

  for i in tqdm(range(start_index,final_index)): 
    
    if (i!=0)&(i%99==0):
      time.sleep(1.5)
      print('{}th : '.format(i), backtrans)
      np.save(path+'eng_to_kor_train_{}_{}.npy'.format(start_index,final_index),trans_list)
    
    try:
      driver.get('https://papago.naver.com/?sk=en&tk='+transed_lang+'&st='+transed_list[i])
      time.sleep(2)
      element=WebDriverWait(driver, 10).until(target_present)
      time.sleep(0.2)
      backtrans = element.text 

      if (backtrans=='')|(backtrans==' '):
        element=WebDriverWait(driver, 10).until(target_present)
        backtrans = element.text 
        trans_list.append(backtrans)
      else:
        trans_list.append(backtrans)
    
    except:
      trans_list.append('')

In [ ]:
trans_list=[]
trans_to_kor(data['eng_title'],'ko',0,2000)
np.save(path+'eng2kor_train_0_2000.npy',trans_list)

  0%|          | 0/2000 [00:00<?, ?it/s]

99th :  원주 중앙시장 화재 현장.
198th :  250년 전, 서울에는 술집이 매우 많았습니다.
297th :  나는 그 집에 IS 괴물이 있는지 몰랐어.마을 주민들도 놀랐습니다.
396th :  제6회 폴란드 코리아 페스티벌.
495th :  삼광글라스는 지난해 118억원의 손실을 봤다.적자를 반전시키다.
594th :  감기에 걸린 구자철이 분데스리가 전반기 마지막 경기에 결장했다.
693th :  화웨이 노바라이트2와 미디어패드 T310 결합 제품이 출시됐다.
792th :  농업 공약 홍준표.
891th :  103만 명이 최고 금리 인하 혜택을 받고 있습니다.안전망 대출 문 확대.
990th :  이계문, 서민을 위한 재원 부족...정부와 금융회사의 금융지원을 늘려야 합니다.
1089th :  순결의 세계가 어떻게 파괴되는지...신기한 공산주의자.
1188th :  국내 정일문 대표가 차별화된 해외주식 서비스를 개발하고 있다.
1287th :  출연연들은 간접고용 노동자를 정규직으로 전환할 필요가 있다.
1386th :  시비를 걸던 술 취한 50대 남성이 자신을 밀쳐 숨지는 사건이 발생했다.30대에게 징역 2년 선고.
1485th :  쌍용 자동차의 2016년형 코란도 투리스모 플러스가 출시되었습니다.
1584th :  더불어민주당이 강릉 지역구 사전투표함에 대한 투명한 관리를 요구하고 나섰다.
1683th :  미국 규제에 대한 철강주의 엇갈린 반응…영향 제한 분석은 또한...
1782th :  최태웅 감독, 팀에 대한 전광인의 기여도가 절반이 넘는다.
1881th :  이란 혁명수비대, 걸프해에 외국 유조선 억류
1980th :  강백호와 박치국의 올스타전 투타 활약이 돋보인다.


In [ ]:
trans_list=[]
trans_to_kor(data['eng_title'], 'ko',2000,4000)
np.save(path+'eng2kor_train_2000_4000.npy',trans_list)

  0%|          | 0/2000 [00:00<?, ?it/s]

2079th :  서해는 날씨가 좋지 않다.인천에서 백령으로 가는 3개 항로를 통제한다.
2178th :  서울옥션해외문화재단의 문화재 환수를 위해 이들이 손을 맞잡았다.
2277th :  프로배구 FA 협상 10일째인 이날 남자부 0계약 발표 이유는 다음과 같다.
2376th :  중국 선양서 설립 100주년 기념식.
2475th :  자존심을 회복한 김현수, 황재균 씨...박병호는 복귀하기로 결심했다.
2574th :  우리 군의 장기 검토가 이뤄지는 남북 정상회담이 열린다.
2673th :  미-터키 정상회담. 다시 전화...에르도안 총리는 시리아 북부를 장악할 준비가 돼 있다.
2772th :  카카오 인수 소식에 특색 있는 종목들이 급등하고 있다.
2871th :  남자 배구는 3강전에서 굳어졌다.선두 다툼은 혼돈스러운 싸움이다.
2970th :  페이스북과 인스타그램은 죽었어요글로벌 SNS 이용자들은 혼란스러워하고 있다.
3069th :  프로농구 FA 명단에서 4명을 제외하고...1일부터 46명이 원 소속팀과 협상합니다.
3168th :  이재용 공개, 비트코인 급락 등 관심이 쏠리고 있다.
3267th :  문 대통령, 올림픽을 유치해 주셔서 감사합니다.우리가 MB를 성공적으로 마무리했으면 좋겠습니다.
3366th :  동양 보통주 1주당 50원 현금배당.
3465th :  또한 헌법 제1조에 언급된 유승민.권력은 사람들로부터 나온다.
3564th :  FIFA 2026 월드컵의 48개 참가국들로 가득 차 있다.
3663th :  문 대통령이 내일 미국을 방문한다. 악화된 날로 북미 중재의 분수령이 펼쳐지고 있다.
3762th :  내년은 코리안 심포니 창단 35주년이다.베토벤 대신에 그것을 하지 마세요.
3861th :  15개 지역에서 2018 가을걷기투어 축제가 열린다.
3960th :  미국과 중국을 상대로 해안경비대를 적극 활용한다...하와이랑 괌. 배 3척.


In [ ]:
trans_list=[]
trans_to_kor(data['eng_title'], 'ko',4000,6000)
np.save(path+'eng2kor_train_4000_6000.npy',trans_list)

  0%|          | 0/2000 [00:00<?, ?it/s]

4059th :  KBO초등학교와 리틀야구단이 공급업체 선정 입찰에 나선다.
4158th :  
4257th :  KT가 판교제로시티 자율주행 실증단지를 조성한다.
4356th :  MBC 뉴스데스크는 "가자" 코너를 신설했다.
4455th :  수정안이 26일 제출됐습니다6월 개헌 투표 공약을 지키며 국회의 동의를 촉구한다.
4554th :  여자농구 삼성생명 KEB하나은행이 4쿼터 역전승을 거뒀다.3연패를 면하세요.
4653th :  저커버그 카메라는 증강현실 플랫폼이다.
4752th :  런던 심포니의 천생연분...평화롭지만 지루해요.
4851th :  류중탁 배구협회 회장 선임
4950th :  미네소타 대학에서 불교 철학에 대한 강의.
5049th :  민간기관을 겨냥한 사이버 공격 건수가 4년 새 3배 증가했다.
5148th :  SK 메이스가 프로농구 챔피언 1차전에서 동료 선수들에게 사과했다.
5247th :  기아차 임직원을 대상으로 여름휴가 대신 말라위에서 봉사활동을 펼친다.
5346th :  민주적 추상화 같은 국가 질서에 대한 책임...평화와 민생.
5445th :  지금이 부동산 시장의 봄 성수기인가요?주말은 전국적으로 견본주택으로 붐빈다.
5544th :  박근혜 대통령님, 선배 과학자 발전의 뿌리가 돼 주셔서 감사드린다.
5643th :  가습기특별위원회 청문회 절반...야당의 여당 부재. 의견 차이.
5742th :  많은 일본 투자와 높은 품질의 투자 대. 중국의 사익을 추구하지 말라...아프리카 신경전.
5841th :  사인이 켜지면 자동으로 번역해주는 구글 워드렌즈.
5940th :  미국에서 12월 금리 인상 예고로 원·달러 환율이 급등했다.


In [ ]:
trans_list=[]
trans_to_kor(data['eng_title'], 'ko',6000,8000)
np.save(path+'eng2kor_train_6000_8000.npy',trans_list)

  0%|          | 0/2000 [00:00<?, ?it/s]

6039th :  아이폰 11이 오늘 출시되었습니다.LTE폰이 계속해서 기대 이상의 성과를 거둘 수 있을까요?
6138th :  시진핑이 이달 말 바티칸을 방문해 교황을 만날 가능성도 거론된다.
6237th :  KT 스마트폰 배터리 감축 기술은 평균 38%가량 줄었다.
6336th :  GC녹십자는 2분기 영업이익이 196억원 47.5% 증가했다.
6435th :  중국 창어 4호 인류 첫 달 뒷면 착륙...우주를 과시하는 것.
6534th :  상반기 정기 인사말입니다여성을 주요 위치에 배치하세요.
6633th :  여당인 호남 대신 수도권을 줄이기 위해 정부가 기준일 변경을 검토하고 있다.
6732th :  빌 클린턴, 이번엔 힐러리에 대한 성폭행 주장 2005.08
6831th :  한국 경제의 글로벌 요인에 금융연구소가 흔들릴 수 있다.
6930th :  윤병세 북한 외무상이 대북제재 협력외교를 비판한다.
7029th :  한국광고협회 광고주 상생 세미나
7128th :  2018 코오롱 그룹 프리뷰.
7227th :  북한 탁구팀이 돌아오다.
7326th :  손학규 하태경 바른미래당 의원 15명이 부당한 징계를 고수할 때 중대한 결단을 내린다.
7425th :  북측 근로자소송법원의 소송위임 적법성 시정요청.
7524th :  권창훈의 풀타임 무대...디종 보르도를 상대로 32 역전승을 거두었습니다.
7623th :  손병석 차관이 인사하고 있습니다.
7722th :  델타 항공, 애틀랜타로 무료로 여행가세요.
7821th :  월드컵 손흥민과 황희찬이 로사노를 떠난다. 리우의 승리를 재현해 봅시다.
7920th :  인터넷


In [ ]:
trans_list=[]
trans_to_kor(data['eng_title'], 'ko',8000,10000)
np.save(path+'eng2kor_train_8000_10000.npy',trans_list)

  0%|          | 0/2000 [00:00<?, ?it/s]

8019th :  북한 노동신문이 협상을 진행하려는 미국의 제재는 독자적으로 처리해야 한다.
8118th :  그는 프로 축구에서 아산으로의 승격이 박탈될 것이다.성남의 1부 복귀 첫 낭독.
8217th :  Cho Land vs. 파이팅. 사직하다. 포털 검색어 대회.
8316th :  루터키 소치서 3자 정상회담...시리아 문제 해결 논의.
8415th :  참엔지니어링이 자본금을 113억원 늘리기로 했다.
8514th :  문재인 대통령 같은 나라를 만들기 위해서는 개헌에 박차를 가해야 한다.지금이 적기야.
8613th :  라니에리는 레스터 시티를 우승한 지 9개월 만에 경질되었다.
8712th :  민주, 안철수 신당 인재, 호남 인사들이 당권경쟁을 벌이고 있다.
8811th :  노트북, 태블릿, 그리고 S펜은...갤럭시북이 오늘 출시되었습니다.
8910th :  배터리가 고장 나도 폭발하지 않습니다.
9009th :  송민순의 외교부 회고록에 대해서는 언급할 것이 없다.
9108th :  사우디아라비아 외국인들에게 처음으로 영주권을 발급했습니다.2억 5천만 원.
9207th :  모든 예술가를 위한 축제...베니스 비엔날레가 13일 개막한다.
9306th :  미국의 백인 노동자들이 사다리를 오르는 것이 어려운 이유는
9405th :  허정재 U-17 여자 축구 감독은 최악의 조에 속해 있다.월드컵 8강.
9504th :  연승은 없으니 연패는...전자랜드, 할로웨이 변수를 넘어서
9603th :  
9702th :  이승우의 베로나 첫 훈련...21번이 배정되었습니다.
9801th :  한국수출입은행이 개성공단 경협보험 가입 접수를 시작했다.
9900th :  나사는 내년 8월에 태양 탐사선을 발사합니다.
9999th :  민간 위성 개발 시대가 열릴 것이다.KAI 중형위성 2호 개발 관리.


In [ ]:
back0=np.load(path+'eng2kor_train_0_2000.npy')
back1=np.load(path+'eng2kor_train_2000_4000.npy')
back2=np.load(path+'eng2kor_train_4000_6000.npy')
back3=np.load(path+'eng2kor_train_6000_8000.npy')
back4=np.load(path+'eng2kor_train_8000_10000.npy')
back_train_fin=[*back0,*back1,*back2,*back3,*back4]
back_train_fin=pd.DataFrame(back_train_fin,columns=['back_title'])

In [ ]:
back_train_fin

,back_title
0,19시간 동안 조사했던 손석희와 김웅...공격과 협박에 대한 평행선.
1,진중권이 돌아보는 조영남 그림의 대작.
2,"청와대 경호실 차장, 비밀리에 방미...미국의 선제타격론에 관심이 이어지고 있다."
3,마스크와 우산을 꼭 챙기세요. 미세먼지와 빗방울.
4,KT 연패의 일등공신 내쉬가 매우 절실한 승리였다.
...,...
9995,황 총리는 테러 가능성이 있는 시설에서 테러 대비태세를 점검한다.
9996,야권통합을 위한 특사여야 한다.당신은 꼼꼼히 검토해야 합니다.
9997,미 국무부의 881개의 이메일이 추가로 공개될 예정입니다.88건은 기밀 사항입니다
9998,민간 위성 개발 시대가 열릴 것이다.KAI 중형위성 2호 개발 관리.


In [ ]:
data

,index,title,topic_idx,topic_name,eng_title
0,10000,19시간 조사받은 손석희·김웅…폭행·협박 놓고 평행선,2,사회,Son Seokhee and Kim Woong who were surveyed fo...
1,10001,진중권이 돌아보는 조영남 그림 대작 사건,3,생활문화,The masterpiece of Cho Youngnam's painting tha...
2,10002,靑안보실 차장 비공개 방미…美일각 선제타격론 제기속 주목,6,정치,Deputy Director of the Blue House Security Off...
3,10003,마스크·우산 챙기세요 미세먼지에 빗방울도,3,생활문화,Make sure to bring a mask and an umbrella. Fin...
4,10004,kt 연패 탈출 1등 공신 내쉬 아주 절실한 승리였다,5,스포츠,The first place contributor to KT's consecutiv...
...,...,...,...,...,...
9995,19995,황총리 IS 테러가능성 있는 시설물 테러대비 태세 점검,6,정치,Prime Minister Hwang checks the readiness for ...
9996,19996,野 국민통합 위한 특사돼야…신중한 검토 필요,6,정치,It should be a special envoy for national inte...
9997,19997,미 국무부 힐러리 이메일 881건 추가공개…88건 기밀,4,세계,U.S. Department of State Hillary's 881 additio...
9998,19998,민간 위성개발 시대 열린다…KAI 중형위성 2호기 개발 주관,0,IT과학,The era of civilian satellite development will...


In [ ]:
train_fin_12=pd.concat([data,back_train_fin],axis=1)
train_fin_12

,index,title,topic_idx,topic_name,eng_title,back_title
0,10000,19시간 조사받은 손석희·김웅…폭행·협박 놓고 평행선,2,사회,Son Seokhee and Kim Woong who were surveyed fo...,19시간 동안 조사했던 손석희와 김웅...공격과 협박에 대한 평행선.
1,10001,진중권이 돌아보는 조영남 그림 대작 사건,3,생활문화,The masterpiece of Cho Youngnam's painting tha...,진중권이 돌아보는 조영남 그림의 대작.
2,10002,靑안보실 차장 비공개 방미…美일각 선제타격론 제기속 주목,6,정치,Deputy Director of the Blue House Security Off...,"청와대 경호실 차장, 비밀리에 방미...미국의 선제타격론에 관심이 이어지고 있다."
3,10003,마스크·우산 챙기세요 미세먼지에 빗방울도,3,생활문화,Make sure to bring a mask and an umbrella. Fin...,마스크와 우산을 꼭 챙기세요. 미세먼지와 빗방울.
4,10004,kt 연패 탈출 1등 공신 내쉬 아주 절실한 승리였다,5,스포츠,The first place contributor to KT's consecutiv...,KT 연패의 일등공신 내쉬가 매우 절실한 승리였다.
...,...,...,...,...,...,...
9995,19995,황총리 IS 테러가능성 있는 시설물 테러대비 태세 점검,6,정치,Prime Minister Hwang checks the readiness for ...,황 총리는 테러 가능성이 있는 시설에서 테러 대비태세를 점검한다.
9996,19996,野 국민통합 위한 특사돼야…신중한 검토 필요,6,정치,It should be a special envoy for national inte...,야권통합을 위한 특사여야 한다.당신은 꼼꼼히 검토해야 합니다.
9997,19997,미 국무부 힐러리 이메일 881건 추가공개…88건 기밀,4,세계,U.S. Department of State Hillary's 881 additio...,미 국무부의 881개의 이메일이 추가로 공개될 예정입니다.88건은 기밀 사항입니다
9998,19998,민간 위성개발 시대 열린다…KAI 중형위성 2호기 개발 주관,0,IT과학,The era of civilian satellite development will...,민간 위성 개발 시대가 열릴 것이다.KAI 중형위성 2호 개발 관리.


In [ ]:
emp_ind0=[]
for i in range(10000):
  if train_fin_12['eng_title'][i] =="":
    emp_ind0.append(i)

In [ ]:
emp_ind0

[]

In [ ]:
train_fin_12.isnull().sum()

index         0
title         0
topic_idx     0
topic_name    0
eng_title     0
back_title    0
dtype: int64

In [ ]:
train_fin_12.to_csv("train_kor2eng_tot.csv", header=True, index=False)

In [ ]:
train_fin_12

,index,title,topic_idx,topic_name,eng_title,back_title
0,10000,19시간 조사받은 손석희·김웅…폭행·협박 놓고 평행선,2,사회,Son Seokhee and Kim Woong who were surveyed fo...,19시간 동안 조사했던 손석희와 김웅...공격과 협박에 대한 평행선.
1,10001,진중권이 돌아보는 조영남 그림 대작 사건,3,생활문화,The masterpiece of Cho Youngnam's painting tha...,진중권이 돌아보는 조영남 그림의 대작.
2,10002,靑안보실 차장 비공개 방미…美일각 선제타격론 제기속 주목,6,정치,Deputy Director of the Blue House Security Off...,"청와대 경호실 차장, 비밀리에 방미...미국의 선제타격론에 관심이 이어지고 있다."
3,10003,마스크·우산 챙기세요 미세먼지에 빗방울도,3,생활문화,Make sure to bring a mask and an umbrella. Fin...,마스크와 우산을 꼭 챙기세요. 미세먼지와 빗방울.
4,10004,kt 연패 탈출 1등 공신 내쉬 아주 절실한 승리였다,5,스포츠,The first place contributor to KT's consecutiv...,KT 연패의 일등공신 내쉬가 매우 절실한 승리였다.
...,...,...,...,...,...,...
9995,19995,황총리 IS 테러가능성 있는 시설물 테러대비 태세 점검,6,정치,Prime Minister Hwang checks the readiness for ...,황 총리는 테러 가능성이 있는 시설에서 테러 대비태세를 점검한다.
9996,19996,野 국민통합 위한 특사돼야…신중한 검토 필요,6,정치,It should be a special envoy for national inte...,야권통합을 위한 특사여야 한다.당신은 꼼꼼히 검토해야 합니다.
9997,19997,미 국무부 힐러리 이메일 881건 추가공개…88건 기밀,4,세계,U.S. Department of State Hillary's 881 additio...,미 국무부의 881개의 이메일이 추가로 공개될 예정입니다.88건은 기밀 사항입니다
9998,19998,민간 위성개발 시대 열린다…KAI 중형위성 2호기 개발 주관,0,IT과학,The era of civilian satellite development will...,민간 위성 개발 시대가 열릴 것이다.KAI 중형위성 2호 개발 관리.
